# RaaGAN

a GAN based filter method to substruct noises from a radar signal

### 1) Data pipeline

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import scipy.io.wavfile as wavfile
from io import *
from pathlib import Path

In [91]:
# define the path to the dataset
p = Path('.\Dataset\clean')
for file in p.glob('*.wav'):
    print(file)
    break

Dataset\clean\DR1_FCJF0_SA1.wav


In [92]:
# check the data --> to delete
for file in p.glob('*.wav'):
    fm, data = wavfile.read(file)
    #tf.audio.decode_wav(file,desired_channels=-1,desired_samples=16000)
    print(f"fm: {fm}, wav_data: {data}, len_data: ", len(data))
    break

fm: 16000, wav_data: [ 1 -1  2 ... -1 -5 -8], len_data:  46797


In [93]:
# read each file and 
def read_and_slice(filename, wav_canvas_size, stride=0.5):
    fm, wav_data = wavfile.read(filename)
    if fm != 16000:
        raise ValueError('Sampling rate is expected to be 16kHz!')
    signals = slice_signal(wav_data, wav_canvas_size, stride)
    return signals

In [102]:
def slice_signal(signal, window_size, stride=0.5):
    """ Return windows of the given signal by sweeping in stride fractions
        of window
    """
    assert signal.ndim == 1, signal.ndim
    n_samples = signal.shape[0]
    print("n_samples: ", n_samples)
    offset = int(window_size * stride)
    slices = []
    for beg_i in range(0, n_samples, offset):
        end_i = beg_i + offset
        slice_ = signal[beg_i:end_i]
        # pad the rest of the slice with 0
        if end_i - beg_i > n_samples - beg_i:
            slice_ = signal[beg_i:n_samples]
            test_l = np.zeros((offset-(n_samples-beg_i)))
            slice_ = np.concatenate((slice_,test_l),axis=None)

        if slice_.shape[0] == offset:
            slices.append(slice_)
    return np.array(slices, dtype=np.int32)

In [109]:
signals_clean = []
for file in p.glob('*.wav'):
    audio_serial = read_and_slice(file, 2 ** 14, stride=1)
    print(audio_serial.shape)
    signals_clean.append(audio_serial)
    if len(signals_clean) > 9:
        break
    
print(len(signals_clean))

n_samples:  46797
(3, 16384)
n_samples:  34509
(3, 16384)
n_samples:  49460
(4, 16384)
n_samples:  45466
(3, 16384)
n_samples:  57856
(4, 16384)
n_samples:  24679
(2, 16384)
n_samples:  27751
(2, 16384)
n_samples:  23143
(2, 16384)
n_samples:  36250
(3, 16384)
n_samples:  39220
(3, 16384)
10
